### Importando Bibliotecas

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from flask import Flask, jsonify

### Definição dos ativos financeiros (fundos e criptomoedas)

In [ ]:
tickers_fundos = ["BOVA11.SA", "IVVB11.SA", "HASH11.SA", "SMAL11.SA", "XFIX11.SA", "AAPL", "MSFT", "TSLA", "GOOGL", "AMZN"]
tickers_criptos = ["BTC-USD", "ETH-USD", "ADA-USD", "XRP-USD", "SOL-USD"]
tickers = tickers_fundos + tickers_criptos

### Dicionário contendo as URLs dos logos dos ativos

In [ ]:
logos = {
    "BOVA11.SA": "https://logourl.com/bova11.png",
    "IVVB11.SA": "https://logourl.com/ivvb11.png",
    "HASH11.SA": "https://logourl.com/hash11.png",
    "SMAL11.SA": "https://logourl.com/smal11.png",
    "XFIX11.SA": "https://logourl.com/xfix11.png",
    "AAPL": "https://logo.clearbit.com/apple.com",
    "MSFT": "https://logo.clearbit.com/microsoft.com",
    "TSLA": "https://logo.clearbit.com/tesla.com",
    "GOOGL": "https://logo.clearbit.com/google.com",
    "AMZN": "https://logo.clearbit.com/amazon.com",
    "BTC-USD": "https://www.cryptocompare.com/media/37746251/btc.png",
    "ETH-USD": "https://www.cryptocompare.com/media/37746238/eth.png",
    "ADA-USD": "https://www.cryptocompare.com/media/37746235/ada.png",
    "XRP-USD": "https://www.cryptocompare.com/media/37746242/xrp.png",
    "SOL-USD": "https://www.cryptocompare.com/media/37746243/sol.png"
}

### Função para baixar os dados históricos

In [ ]:
def obter_dados_mercado(tickers, data_inicio="2020-01-01", data_fim="2025-12-31"):
    data = yf.download(tickers, start=data_inicio, end=data_fim)
    df = data[['Open', 'High', 'Low', 'Close', 'Volume']].stack(level=1).reset_index()
    df.columns = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
    df["Logo_URL"] = df["Ticker"].map(logos)
    return df

df = obter_dados_mercado(tickers)

### Função para calcular métricas financeiras

In [ ]:
def calcular_metricas(df):
    df['Daily Change (%)'] = ((df['Close'] - df['Open']) / df['Open']) * 100
    df['Weekly Change (%)'] = df.groupby('Ticker')['Close'].pct_change(periods=5) * 100
    df['Monthly Change (%)'] = df.groupby('Ticker')['Close'].pct_change(periods=21) * 100
    df['SMA_7'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=7).mean())
    df['SMA_30'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=30).mean())
    df['EMA_7'] = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=7, adjust=False).mean())
    df['EMA_30'] = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=30, adjust=False).mean())
    df['Vol_Media_7'] = df.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(window=7).mean())
    df['Vol_Media_30'] = df.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(window=30).mean())
    df['Liquidez'] = df['Vol_Media_30']
    return df

df = calcular_metricas(df)

### Função para cálculo do RSI

In [ ]:
def calcular_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df['RSI_14'] = df.groupby('Ticker')['Close'].transform(lambda x: calcular_rsi(x, period=14))

### Função para coletar informações das empresas

In [ ]:
def obter_dados_empresa(tickers):
    dados_empresa = []
    for ticker in tickers:
        empresa = yf.Ticker(ticker)
        info = empresa.info
        dados_empresa.append([
            ticker,
            info.get("longName", ticker),
            info.get("sector", "N/A"),
            info.get("CEO", "N/A"),
            info.get("address1", "N/A"),
            info.get("fullTimeEmployees", "N/A"),
            info.get("longBusinessSummary", "N/A"),
            info.get("exchange", "N/A")
        ])
    return pd.DataFrame(dados_empresa, columns=[
        "Ticker", "Nome", "Setor", "CEO", "Endereço", "Número de Funcionários", "Descrição", "Bolsa"
    ])

df_empresa = obter_dados_empresa(tickers)
df = df.merge(df_empresa, on="Ticker", how="left")

### Criando API Flask

In [ ]:
app = Flask(__name__)

@app.route('/dados', methods=['GET'])
def get_data():
    return jsonify(df.fillna('').to_dict(orient='records'))

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)